In [1]:
!nvidia-smi

Sun Dec 10 05:33:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.04              Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0  On |                  Off |
|  0%   33C    P8              27W / 450W |   2027MiB / 24564MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Merge Model and Lora

Adapted from https://github.com/brevdev/notebooks/blob/main/gguf-export.ipynb

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os
import torch
import argparse

model_name = "mistralai/Mistral-7B-v0.1"
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

model = PeftModel.from_pretrained(base_model, "mistral-discern-finetune/checkpoint-11000")
 
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)
 
model.save_pretrained("./merged_model")
tokenizer.save_pretrained("./merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/tokenizer.model',
 './merged_model/added_tokens.json',
 './merged_model/tokenizer.json')

## Convert to GGUF

Probably need to restart the kernel here

In [2]:
!apt update
!apt install curl -y 
!curl -L -o convert.py https://github.com/ggerganov/llama.cpp/raw/master/convert.py
!curl -L -o requirements.txt https://github.com/ggerganov/llama.cpp/raw/master/requirements.txt

!pip install -r requirements.txt

Hit:1 http://deb.debian.org/debian bullseye InRelease
Hit:2 http://deb.debian.org/debian-security bullseye-security InRelease
Hit:3 http://deb.debian.org/debian bullseye-updates InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
1 package can be upgraded. Run 'apt list --upgradable' to see it.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.74.0-1.3+deb11u10).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 50517  100 50517    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dl

In [ ]:
!curl -L -o quantize.pl https://github.com/ggerganov/llama.cpp/raw/master/requirements.txt

In [1]:
# Convert the 7B model to ggml FP16 format
!python3 convert.py merged_model

Loading model file merged_model/model-00001-of-00003.safetensors
Loading model file merged_model/model-00001-of-00003.safetensors
Loading model file merged_model/model-00002-of-00003.safetensors
Loading model file merged_model/model-00003-of-00003.safetensors
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=32768, n_ff=14336, n_head=32, n_head_kv=8, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=None, path_model=PosixPath('merged_model'))
Loading vocab file 'merged_model/tokenizer.model', type 'spm'
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting layer 20
Permuting

In [ ]:
# # [Optional] for models using BPE tokenizers
# !python convert.py model --vocabtype bpe

In [2]:
# # [Optional] quantize the model to 4-bits (using q4_0 method)
# Will need llama.cpp installed!
#!./quantize ./merged_model/ggml-model-f16.gguf ./merged_model/ggml-model-q4_0.gguf q4_0

!docker run --gpus all -v .:/models -it ghcr.io/ggerganov/llama.cpp:full-cuda --quantize /models/merged_model/ggml-model-f16.gguf /models/merged_model/ggml-model-q4_0.gguf q4_0

/bin/sh: 1: ./quantize: not found


In [ ]:
# # [Optional] update the gguf filetype to current if older version is unsupported by another application
# !./quantize ./model/ggml-model-q4_0.gguf ./model/ggml-model-q4_0-v2.gguf COPY